## Table of Contents
1. [The PyCrypto Library](#pycrypto)
2. [Exercise 1. Advanced Encryption Standard](#aes)
  1. [AES-ECB](#1.1-AES-ECB)
  2. [AES-ECB](#1.1-AES-CBC)
  2. [AES-ECB](#1.2-AES-CTR)
3. [Exercise 2: ChaCha20](#chacha)

## The PyCrypto Libarary <a name='pycrypto'/>

Python offers a library called [Cryptography](https://cryptography.io) which includes various cryptographic algorithms, among them AES.

<b> If you're using the Docker images, the library is already installed on your image, otherwise you install it using *pip* </b>

In [ ]:
pip install pycrypto --user

A documentation of the library's API is found [here](https://cryptography.io/en/latest/hazmat/primitives/))

Make yourself familar with the library go with the following exercises

## Excercise 1: Advanced Encryption Standard <a name="aes"/>

Goal of this exercise is to make yourself familar with AES and how to use the different modes of AES, namely ECB,CBC and CTR.


### 1.1 AES-ECB
In the following code block, a variable 100x200 *array* is filled with values in the range 0 to 255, and thereby represents an image. You can plot the image by executing the following cell.

*Note: you may have to install numpy, matplotlib for this exercise*

In [ ]:
pip install numpy matplotlib --user

In [ ]:
import numpy as np

array = np.zeros([128, 128], dtype=np.uint8)

# Set grey value to black or white depending on x position
for x in range(128):
    for y in range(128):
        if (x % 16) // 8 == (y % 16) // 8:
            array[y, x] = 0
        else:
            array[y, x] = 255

In [ ]:
import matplotlib.pyplot as plt
import PIL.Image

img = PIL.Image.fromarray(array)
im_array = np.asarray(img)
plt.imshow(im_array)
plt.show()

#### Encrypt
- Use *AES-ECB* with a key of your choice to encrypt the values stored in *array*. Plot the array again and experience how -- although encrypted -- the image is still readable.
- Try it with all three available key sizes (16,24,32 byte) and compare the difference

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = b'Sixteen byte key'
# iv = Random.new().read(AES.block_size)
#cipher = AES.new(key, AES.MODE_ECB)
cipher = Cipher(algorithms.AES(key),modes.ECB()).encryptor()


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = b'Sixteen byte key'
# iv = Random.new().read(AES.block_size)
#cipher = AES.new(key, AES.MODE_ECB)
cipher = Cipher(algorithms.AES(key),modes.ECB()).encryptor()
enc = cipher.update(bytes(array))
#encrypt(bytes(array))
img_enc=[]
for i in range(100):
    img_enc.append(list(enc)[i*100:(i+1)*100])


In [ ]:
img_enc = np.array(img_enc, dtype=np.uint8)
img = PIL.Image.fromarray(img_enc)
im_array = np.asarray(img)
plt.imshow(im_array)
plt.show()

#### 1.2 AES-CBC
- Now try the same with AES-CBC and experience the difference. Make sure that you choose a random Initialization Vector
- Try it with all three available key sizes (16,24,32 byte) and compare the difference

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

key = b'Sixteen byte key'
iv = os.urandom(len(key))
#cipher = AES.new(key, AES.MODE_ECB)
cipher = Cipher(algorithms.AES(key),modes.CBC(iv)).encryptor()

enc = cipher.update(bytes(array))
img_enc=[]
for i in range(100):
    img_enc.append(list(enc)[i*100:(i+1)*100])
img_enc = np.array(img_enc, dtype=np.uint8)

In [ ]:
img = PIL.Image.fromarray(img_enc)
im_array = np.asarray(img)
plt.imshow(im_array)
plt.show()

#### 1.3 AES-CTR
The goal of this part is to experience the problem when a counter is used twice in *AES-CTR* mode
1. Encrypt the text *Biden is the next president of the US* with AES-CTR and a randomly chosen 16byte key
2. Encrypt the text *Trump stays president of the US* with AES-CTR **with the same key**
3. Use the same Counter for both encryption
4. XOR the resulting ciphertext and compare them with the XOR of the two plaintexts


In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

textA = b'Trump is the next president of the US'
textB = b'Biden stays president of the US'

iv = os.urandom(len(key))

cipherA = Cipher(algorithms.AES(key),modes.CTR(b'1'*len(key))).encryptor()
cipherB = Cipher(algorithms.AES(key),modes.CTR(b'1'*len(key))).encryptor()

encA = cipherA.update(textA)
encB = cipherB.update(textB)

In [ ]:
print(bytes(a ^ b for (a, b) in zip(textA,textB)))
print(bytes(a ^ b for (a, b) in zip(encA,encB)))

## Exercise 2: ChaCha20 <a name="chacha"/>
Try to implement ChaCha20 by yourself. For correctness use the test vectors given by [RFC7539](https://tools.ietf.org/html/rfc7539)
1. [ChaCha20 quarter round](https://tools.ietf.org/html/rfc7539#section-2.1)

In [1]:
def plus(ina,inb):
    x = int.from_bytes(ina, byteorder='big')+int.from_bytes(inb, byteorder='big')
    return (x & 0xFFFFFFFF).to_bytes(4, 'big')

def xor(ina,inb):
    return bytes(a^b for (a, b) in zip(ina,inb))

def rotl(a,b):
    x = int.from_bytes(a, byteorder='big')
    x =(((x) << (b)) | ((x) >> (32 - (b))))
    return (x & 0xFFFFFFFF).to_bytes(4, 'big')

#Quarter Round Function
def qr(a, b, c, d):
    a = plus(a,b)
    d = xor(a,d)
    d = rotl(d,16)
    c = plus(c,d)
    b = xor(b,c)
    b = rotl(b,12)
    a = plus(a,b)
    d = xor(a,d)
    d = rotl(d,8)
    c = plus(c,d)
    b = xor(b,c)
    b = rotl(b,7)
    return (a,b,c,d)

In [2]:
#Tests for Quarter Round

a = bytes.fromhex("11111111")
b = bytes.fromhex("01020304")
c = bytes.fromhex("9b8d6f43")
d = bytes.fromhex("01234567")

(a2,b2,c2,d2) = qr(a,b,c,d)
print(a.hex(),'->',a2.hex())
print(b.hex(),'->',b2.hex())
print(c.hex(),'->',c2.hex())
print(d.hex(),'->',d2.hex())

11111111 -> ea2a92f4
01020304 -> cb1cf8ce
9b8d6f43 -> 4581472e
01234567 -> 5881c4bb



2. [ChaCha20 block function](https://tools.ietf.org/html/rfc7539#section-2.3.2)

In [ ]:
# ChaCha Rounds
# Input State =  cccccccc  cccccccc  cccccccc  cccccccc
#                kkkkkkkk  kkkkkkkk  kkkkkkkk  kkkkkkkk
#                kkkkkkkk  kkkkkkkk  kkkkkkkk  kkkkkkkk
#                bbbbbbbb  nnnnnnnn  nnnnnnnn  nnnnnnnn
#   c=constant k=key b=blockcount n=nonce

# b"expand 32-byte k" from RFC7539
constant = bytes.fromhex("61707865") + bytes.fromhex("3320646e") + bytes.fromhex("79622d32") + bytes.fromhex("6b206574")

# Helper Method to print the state more nicely
def print_state(state):
    for i in range(4):
        print(state[i*4].hex(),state[i*4+1].hex(),state[i*4+2].hex(),state[i*4+3].hex())


#// Odd round
#QR(0, 4,  8, 12)	// 1st column
#QR(1, 5,  9, 13)	// 2nd column
#QR(2, 6, 10, 14)	// 3rd column
#QR(3, 7, 11, 15)	// 4th column
#// Even round
#QR(0, 5, 10, 15)	// diagonal 1 (main diagonal)
#QR(1, 6, 11, 12)	// diagonal 2
#QR(2, 7,  8, 13)	// diagonal 3
#QR(3, 4,  9, 14)	// diagonal 4
def inner_block(state):
    (state[0],state[4],state[8],state[12]) = qr(state[0],state[4], state[8],state[12])
    (state[1],state[5],state[9],state[13]) = qr(state[1],state[5], state[9],state[13])
    (state[2],state[6],state[10],state[14]) = qr(state[2],state[6], state[10],state[14])
    (state[3],state[7],state[11],state[15]) = qr(state[3],state[7], state[11],state[15])
    (state[0],state[5],state[10],state[15]) = qr(state[0],state[5], state[10],state[15])
    (state[1],state[6],state[11],state[12]) = qr(state[1],state[6], state[11],state[12])
    (state[2],state[7],state[8],state[13]) = qr(state[2],state[7], state[8],state[13])
    (state[3],state[4],state[9],state[14]) = qr(state[3],state[4], state[9],state[14])

# Prepare init state and run 10 rounds of inner block (20 rounds in total)
def chacha20_block(key, counter, nonce):
    state = constant + key + counter + nonce
    #Initialize the array with the state. It's all binaries
    working_state = []
    for i in range(16):
        working_state.append(state[i*4:(i+1)*4])
    
    #Save the init state, we need it for the last add
    state=working_state.copy()
    
    # Apply the chacha double rounds
    for i in range(10):
        inner_block(working_state)
    
    #add the init state to the result
    for i in range(16):
        working_state[i]=plus(working_state[i],state[i])
    
    out=b''
    for i in range(16):
        out += working_state[i]
    return (out,working_state, state)

Test the ChaCha20_block function with the test parameters from RFC7539

input: 
```
61707865  3320646e  79622d32  6b206574
03020100  07060504  0b0a0908  0f0e0d0c
13121110  17161514  1b1a1918  1f1e1d1c
00000001  09000000  4a000000  00000000
```       
expected output

```
e4e7f110  15593bd1  1fdd0f50  c47120a3
c7f4d1c7  0368c033  9aaa2204  4e6cd4c3
466482d2  09aa9f07  05d7c214  a2028bd9
d19c12b5  b94e16de  e883d0cb  4e3c50a2
```

In [ ]:
# Test
key = bytes.fromhex("03020100070605040b0a09080f0e0d0c13121110171615141b1a19181f1e1d1c")
nonce = bytes.fromhex("090000004a00000000000000")
counter = bytes.fromhex("00000001")

(x,out,init) = chacha20_block(key,counter,nonce)
print_state(init)
print()
print_state(out)


3. [ChaCha20 encryption](https://tools.ietf.org/html/rfc7539#section-2.4.2)

In [ ]:
# Helper method which reorders the bytes in the key stream.
def reorder_keystream(keystream):
    #reoder keystream
    temp = bytearray(keystream)
    for i in range(0,len(keystream),2):
        temp[i] = keystream[i+1]
        temp[i+1] = keystream[i]
    keystream = temp.copy()
    for i in range(0,len(keystream),4):
        temp[i] = keystream[i+2]
        temp[i+1] = keystream[i+3]
        temp[i+2] = keystream[i]
        temp[i+3] = keystream[i+1]
    return temp
        
# create a keystream which is as long as the message and xor with the message.    
def encrypt(key,msg,counter,nonce):
    blocks=int(len(msg)/64)+1
    cipher = b''
    final_keystream=b''
    for i in range(blocks):
        c = (counter+i & 0xFFFFFFFF).to_bytes(4, 'big')
        keystream = chacha20_block(key,c,nonce)[0]
        if (i+1)*32 <= len(msg):
            final_keystream+=keystream
        else:
            final_keystream+=keystream[0:len(msg)]
    
    # we have to reorder the byte-stream in order to match the encoding of the test vectors
    final_keystream = reorder_keystream(final_keystream)
    
    cipher = bytes(a ^ b for (a, b) in zip(msg,final_keystream))
    return cipher

Test message:
```
  000  4c 61 64 69 65 73 20 61 6e 64 20 47 65 6e 74 6c  Ladies and Gentl
  016  65 6d 65 6e 20 6f 66 20 74 68 65 20 63 6c 61 73  emen of the clas
  032  73 20 6f 66 20 27 39 39 3a 20 49 66 20 49 20 63  s of '99: If I c
  048  6f 75 6c 64 20 6f 66 66 65 72 20 79 6f 75 20 6f  ould offer you o
  064  6e 6c 79 20 6f 6e 65 20 74 69 70 20 66 6f 72 20  nly one tip for
  080  74 68 65 20 66 75 74 75 72 65 2c 20 73 75 6e 73  the future, suns
  096  63 72 65 65 6e 20 77 6f 75 6c 64 20 62 65 20 69  creen would be i
  112  74 2e 
```


Ciphertext Sunscreen:
```
  000  6e 2e 35 9a 25 68 f9 80 41 ba 07 28 dd 0d 69 81  n.5.%h..A..(..i.
  016  e9 7e 7a ec 1d 43 60 c2 0a 27 af cc fd 9f ae 0b  .~z..C`..'......
  032  f9 1b 65 c5 52 47 33 ab 8f 59 3d ab cd 62 b3 57  ..e.RG3..Y=..b.W
  048  16 39 d6 24 e6 51 52 ab 8f 53 0c 35 9f 08 61 d8  .9.$.QR..S.5..a.
  064  07 ca 0d bf 50 0d 6a 61 56 a3 8e 08 8a 22 b6 5e  ....P.jaV....".^
  080  52 bc 51 4d 16 cc f8 06 81 8c e9 1a b7 79 37 36  R.QM.........y76
  096  5a f9 0b bf 74 a3 5b e6 b4 0b 8e ed f2 78 5e 42  Z...t.[......x^B
  112  87 4d        
```

In [ ]:
#Test
msg = b'Ladies and Gentlemen of the class of \'99: If I could offer you only one tip for the future, sunscreen would be it.'

print('Message:')
print(msg.hex())
key = bytes.fromhex('03020100070605040b0a09080f0e0d0c13121110171615141b1a19181f1e1d1c')
nonce = bytes.fromhex('000000004a00000000000000')
counter = 1

cipher = encrypt(key,msg,counter,nonce)
print("Cipher:")
print(cipher.hex())


## Exercise 3: OpenSSL <a name="openssl"/>

For a more practical usage of Cryptography, lets try out its usage with [OpenSSL](https://en.wikipedia.org/wiki/OpenSSL)

OpenSSL is known for its broad usage in the internet, most famously for securing web-traffic e.g., for HTTP/s.
However, OpenSSL offers many more functionalities and is actually a software library that contains implementation of various cryptographic protocols and functionalities and is available for various Unix and Windows operating systems. Additionally, it is open source, carefully reviewed by security experts and therefore widely accepted as secure. 

For end-users, openssl provides a [command line interface](https://wiki.openssl.org/index.php/Command_Line_Utilities), so that you can directly interact with the library.

In Juypter Notebooks, you can directly run such command by starting the cells with `%%bash`, e.g. the following command 
1. generates a file `example.txt` with some random conent
2. outputs the contents of the file `example.txt`

In [17]:
%%bash
openssl rand -base64 365 > example.txt
cat example.txt

7i2lIv0PrvxoH/BE3jvzciY3jy5BbvdVdZgv+baU1LyjXwRkt5YRayx8kJbjy00V
pk1e9b4T0Zco1Ixb5b+B4vOBTIYb6tWG2Dl2yuEp6tyRsmjC9Lb9aFt84OGTjsK+
y2fKkEfS5y0dTQstLg+/qYiQ138SZsjXGdsJT/GhvERqPxbRIFUBMDG5yTvejDox
K/TUb85IQ1cTTVjtWLHvW12ed4NJutvaK7X4+z0hXXtSTVXdiXMsFeaRU04a49CI
mYu7RWnr9ISbtpRejd0Pyl/sgA/1xN1+7bsPPFHbV7koPBmSAn36raIji3+OD5AY
iOwd7bOW0JHdnVSehNZGVWFMQnOBGsYRolHV3YBvF2XQj98cHqwM5y/HvSEg7GDc
suD4bycoTN2i4YwbUn8OX8sEszwYhPLeo3ztRhWXcALTNIUAzbGeLUU3Ik4B7LfI
TaGdxOzWbq/dKOcv8mCgQwyuUn2HQDdd5vayzds=


### 3.1 Encrypt a file with AES-256-CBC

Encrypt the file ```example.txt``` with passwort ```CryptoIsalotofFun!```

In [24]:
%%bash
openssl enc -aes-256-cbc --pass pass:CryptoIsalotofFun! -p -in example.txt

*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


Salted__J�v=Y�salt=4A1CE6760E3D599F
key=73C887007766E0832C0711E397CF0E1A56E5193515E233F486271073F98CAD56
iv =0464CCB987AE585825FA7C0B267D81AB
�y�$���� ?p�0�M���Ӵ{��m�!LvC�Rpl���H�W��S��kHXΐUƓO'��_-b����=��_��o<P'[BP�tM*ϴl!�+[�9�����ϩO��l�J� �|�
���;Й�q-���KɃ����$З������f�ܛj�\Nt��7y|b]�$W��d�q�J�Ѯ�a��rz:��w陖Q�q��2�܌�U��ŭ3t���㱸��o�_�_.�>����.=[mʞ�ꑫ�X�D��O��a�E��������B� I@�K4��

Save the output in a file an try to decrypt it again!

In [32]:
%%bash
openssl enc -aes-256-cbc --pass pass:CryptoIsalotofFun! -p -in example.txt -out example.enc

*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


salt=3242F25182E4600B
key=78A9855685B91EC15772FB2122D5BC03C2DAC8FC4EBA9B8C2D5A4687B82B6D55
iv =E749A9E92BEC2192619C556FA3149EE0


In [33]:
%%bash
openssl enc -d -aes-256-cbc --pass pass:CryptoIsalotofFun! -p -in example.enc

*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


salt=3242F25182E4600B
key=78A9855685B91EC15772FB2122D5BC03C2DAC8FC4EBA9B8C2D5A4687B82B6D55
iv =E749A9E92BEC2192619C556FA3149EE0
7i2lIv0PrvxoH/BE3jvzciY3jy5BbvdVdZgv+baU1LyjXwRkt5YRayx8kJbjy00V
pk1e9b4T0Zco1Ixb5b+B4vOBTIYb6tWG2Dl2yuEp6tyRsmjC9Lb9aFt84OGTjsK+
y2fKkEfS5y0dTQstLg+/qYiQ138SZsjXGdsJT/GhvERqPxbRIFUBMDG5yTvejDox
K/TUb85IQ1cTTVjtWLHvW12ed4NJutvaK7X4+z0hXXtSTVXdiXMsFeaRU04a49CI
mYu7RWnr9ISbtpRejd0Pyl/sgA/1xN1+7bsPPFHbV7koPBmSAn36raIji3+OD5AY
iOwd7bOW0JHdnVSehNZGVWFMQnOBGsYRolHV3YBvF2XQj98cHqwM5y/HvSEg7GDc
suD4bycoTN2i4YwbUn8OX8sEszwYhPLeo3ztRhWXcALTNIUAzbGeLUU3Ik4B7LfI
TaGdxOzWbq/dKOcv8mCgQwyuUn2HQDdd5vayzds=


### 3.2 Try with other encryption algorithms as much as you wish :)